In [1]:
from generate import simulate_blackwell_glynn
from nsmm import nsmm_lag1, nsmm_lag1_cate

In [3]:
df = simulate_blackwell_glynn(n_units=1000, t_periods=20)
nsmm_lag1(df, unit = 'i' , treatment = 'X',outcome = 'Y',common_causes= ['Z'])

(-0.0913801121686876, -0.10421430900215296)